In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import acf,pacf
%matplotlib inline


In [63]:
df = pd.read_csv('P2testing.csv',header=0)
Date = pd.to_datetime(df.Date)
JPY = df['JPY.USD']
df.tail()

,Date,AUD.USD,GBP.USD,CAD.USD,NLG.USD,FRF.USD,DEM.USD,JPY.USD,CHF.USD
750,12/24/1998,1.636661,0.597979,1.5515,1.8968,5.6434,1.6840,116.24,1.3760
751,12/28/1998,1.647446,0.597193,1.5525,1.8930,5.6310,1.6793,115.88,1.3720
752,12/29/1998,1.635590,0.593965,1.5520,1.8820,5.5990,1.6707,115.42,1.3630
753,12/30/1998,1.631588,0.600060,1.5492,1.8892,5.6155,1.6765,114.88,1.3805
754,12/31/1998,1.633186,0.601395,1.5375,1.8770,5.5870,1.6670,113.08,1.3735


In [34]:
#Grisha Trubetskoy

def initial_trend(series, L):
    sum = 0.0
    for i in range(L):
        sum += float(series[i+L] - series[i]) / L
    return sum / L

def initial_seasonal_components(series, L):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/L)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[L*j:L*j+L])/float(L))
    # compute initial values
    for i in range(L):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[L*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals

def triple_exponential_smoothing(series, L, alpha, beta, gamma, n_preds):
    result = []
    seasonals = initial_seasonal_components(series, L)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, L)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%L])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%L]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%L] = gamma*(val-smooth) + (1-gamma)*seasonals[i%L]
            result.append(smooth+trend+seasonals[i%L])
    return result


def double_exponential_smoothing(series, alpha, beta):
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # when doing forecast
          value = result[-1]
        else:
          value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

In [60]:
# make prediction
import math
L = 79
alpha = 1
beta = 0.03
gamma = 0.09
n_preds = 1
n =len(JPY)
pred=[]
for i in range(0,n):
    if math.isnan(JPY[i]):
        if i <=2*L:
            data = JPY[0:i]
            result = double_exponential_smoothing(data, alpha, beta)
            JPY[i]=result[-1]
            print result[-1]
            pred.append(JPY[i])
        else:
            data = JPY[0:i]
            result = triple_exponential_smoothing(data,L,alpha,beta,gamma,n_preds)
            JPY[i]=result[-1]
            print result[-1]
            pred.append(JPY[i])


/Users/williammeng/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


105.797890762
107.426640482
107.128140376
104.585580416
105.755932379
106.590785351
108.424661795
106.679238544
104.958650768
107.098994831
109.007687267
108.124439225
110.482653064
107.918017782
106.668510941


/Users/williammeng/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


108.183613867
108.513965092
110.395968734
110.695686157
112.511442681
114.629731458
111.637348723
112.757576807
113.283915411
115.531890935
116.428734454
119.478289009
122.508068328
121.590358767
122.110306536
124.363411573
126.001539277
126.361228006
125.120138042
114.513767475
116.565932049
113.08647472
114.732580522
116.108741357
118.699653646
117.903230012
118.753500281
121.006401297
120.954004134
120.394444223
121.453626457
124.22872302
127.416062653
129.597098422
130.036732143
131.459510745
126.048252507
124.758475918
128.157170259
127.556410031
130.608791626
134.255991951
131.286567155
131.875090489
136.502920793
138.177622995
137.739505644
138.761825615
140.6071613
142.046481631
146.237723308
144.67778159
134.018198936
135.045028486
117.520675475
117.441669892
118.458750869
120.954370801
119.736209815
116.416366643


In [67]:
f= open("Prediction.txt",'w')
f.write("E0178264")
f.write("\n")
for i in range(len(pred)):
    f.write(str(pred[i]))
    f.write("\n")
f.close()

In [68]:
print len(pred)

75
